In [5]:
from influxdb_client import InfluxDBClient
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

## Connessione al database InfluxDB
INFLUXDB_URL = "http://localhost:8086"
TOKEN = os.getenv("token")  
ORG = "IotAlarmSystem"
BUCKET = "Prova"
client = InfluxDBClient(url=INFLUXDB_URL, token=TOKEN, org=ORG)
query_api = client.query_api()

## Query - cambiare i giorni per prendere più o meno dati 
query = f"""
from(bucket: "{BUCKET}")
|> range(start: -90d)
|> filter(fn: (r) => r._measurement == "pressure")
|> filter(fn: (r) => r._field == "value")
|> keep(columns: ["_time", "_value"])
"""

# result = query_api.query(query)
data = []

# for table in result:
#     for record in table.records:
#         data.append({"time": record["_time"], "value": record["_value"]})


# A seconda della fonte dati cambiare l'assegnazioene di df 

# df = pd.DataFrame(data)
df = pd.read_csv("simulated_data_random.csv")

In [ ]:
# start_date = pd.Timestamp("2025-01-20", tz="UTC")
# end_date = pd.Timestamp("2025-01-26", tz="UTC")

def sleep_per_night(df, start_date=None, end_date=None):
    """
    Calcolo tempo totale di sonno per ogni notte, considerando la notte dalle 21:00 del giorno X 
    alle 10:00 del giorno X+1 e ignorando i dati al di fuori di questo intervallo.
    """
    df["time"] = pd.to_datetime(df["time"])
    
    # Filtrare il dataframe se start_date e end_date sono forniti
    if start_date:
        df = df[df["time"] >= start_date]
    if end_date:
        df = df[df["time"] <= end_date]
    
    df["night"] = df["time"].dt.date
    df["hour"] = df["time"].dt.hour

    # Se l'ora è minore di 10, considerare la notte come la notte precedente
    df.loc[df["hour"] < 10, "night"] = df["night"] - pd.Timedelta(days=1)
    df = df[(df["hour"] >= 21) | (df["hour"] < 10)]

    sleep_data = []

    # Calcolare il tempo totale di sonno per ogni notte
    for night, night_df in df.groupby("night"):
        sleep_seconds = 0
        i = 0

        # Scorrere i dati fino a trovare il primo valore uguale a 4095
        while i < len(night_df):
            while i < len(night_df) and night_df.iloc[i]["value"] != 4095:
                i += 1
            if i >= len(night_df):
                break

            start_time = night_df.iloc[i]["time"]

            # Avanzare fino a trovare il primo valore diverso da 4095
            while i < len(night_df) and night_df.iloc[i]["value"] == 4095:
                i += 1
            if i >= len(night_df):
                break

            # Salvare la fine del sonno
            end_time = night_df.iloc[i]["time"]
            duration = (end_time - start_time).total_seconds()
            sleep_seconds += duration

        # Convertire i secondi totali in ore, minuti e secondi
        hours = int(sleep_seconds // 3600)
        minutes = int((sleep_seconds % 3600) // 60)
        seconds = int(sleep_seconds % 60)

        sleep_data.append({"night": night, "hours": hours, "minutes": minutes, "seconds": seconds})

    sleep_summary_df = pd.DataFrame(sleep_data)

    return sleep_summary_df


df_nights = sleep_per_night(df,start_date=None, end_date=None)


## CALCOLO MEDIE
def calculate_average_sleep(df):
    total_avg_seconds = df["hours"].mean() * 3600 + \
                        df["minutes"].mean() * 60 + \
                        df["seconds"].mean()

    # Convertire la media totale in ore, minuti e secondi
    average_hours = int(total_avg_seconds // 3600)
    average_minutes = int((total_avg_seconds % 3600) // 60)
    average_seconds = int(total_avg_seconds % 60)

    print(f"Media Totale: {average_hours} ore, {average_minutes} minuti e {average_seconds} secondi")
    return df

average = calculate_average_sleep(df_nights)
average

Media Totale: 8 ore, 34 minuti e 11 secondi


,night,hours,minutes,seconds
0,2025-01-20,9,21,0
1,2025-01-21,9,29,0
2,2025-01-22,8,27,0
3,2025-01-23,8,9,0
4,2025-01-24,7,59,0
...,...,...,...,...
300,2025-11-16,9,18,0
301,2025-11-17,8,44,0
302,2025-11-18,9,13,0
303,2025-11-19,7,25,0
